## Fx Forward Upsert and Valuation example notebook

This notebook displays the upsert and valuation functionality in Lusid in the context of FX fowards.

In [2]:
from lusidtools.jupyter_tools import toggle_code

"""Fx Forward upsert and valuation in Lusid

Attributes
----------
Transaction Portfolios
Quotes
Holdings
Transactions
Valuation
"""

toggle_code("Toggle Docstring")


In [10]:
import os
import lusid
import lusid.models as lm
from lusid.utilities import ApiClientFactory
from lusid import api
from lusidjam import RefreshingToken
import pandas as pd
import datetime
from datetime import date
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
import uuid
import json
from lusid import ApiException

In [4]:
# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",)

transactions_api= api_factory.build(lusid.TransactionPortfoliosApi)
quotes_api = api_factory.build(lusid.QuotesApi)
instruments_api = api_factory.build(lusid.InstrumentsApi)
configuration_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)


scope='Fwd_Example'

# 1.Create a portfolio

Create a portfolio in Lusid to hold the Fx Forwards. To do this, we use the <a id='https://www.lusid.com/docs/api#tag/Transaction-Portfolios'>Transaction Portfolios API</a>.

In [13]:
def create_portfolio(scope='',display_name='',base_currency='GBP'):
    Port_code='Fx_Port'
    request= lm.CreateTransactionPortfolioRequest(
        display_name=display_name,
        code=Port_code,
        base_currency=base_currency,
        created=str(date(2020,1,1)))
    try:
        result = transactions_api.create_portfolio(
            scope=scope,
            create_transaction_portfolio_request=request)
        return result
    except ApiException as e:
        display(json.loads(e.body)["title"])

In [14]:
Port1_details= create_portfolio(scope='FX_FW_Transact',display_name='Swap_Transaction',base_currency='GBP')
Port1_Code= 'Fx_Port'
display(Port1_Code)

'Could not create a portfolio with id Fx_Port because it already exists in scope FX_FW_Transact.'

'Fx_Port'

## 2. Fund Portfolio with cash

Fund portfolio with initial an cash balance. This is done by upserting a 'FundsIn' transaction of 10000 units of GBP

In [11]:
def fund_portfolio(port_code):
    request=lm.TransactionRequest(
    transaction_id='Intial_funding',
    type='FundsIn',
    instrument_identifiers= {"Instrument/default/Currency":'GBP'},
    transaction_date= str(date(2020,1,1)),
    settlement_date= str(date(2020,1,1)),
    units=10000,
    total_consideration= lm.CurrencyAndAmount(10000,'GBP')
    )
    response = transactions_api.upsert_transactions('FX_FW_Transact',
                                                   Port1_Code,
                                                   transaction_request=[request])
    response= response.to_dict()
    
    return {'response link':response['links'][2]['href']}
       

In [12]:
fund_portfolio(Port1_Code)

{'response link': 'http://omar.lusid.com/app/insights/logs/0HM7R8JIF192E:00000004'}

## 3. Upsert FX Forward position as Transaction

Function to upsert an fx forward position as a transaction to the portfolio created above. This will create
seperate cash balances with holding type F which will be booked at maturity.

In [15]:
def upsert_fx(filename):
    swap_df=pd.read_csv(filename)
    T_id= 'Transaction_1'
    request= lm.TransactionRequest(
    transaction_id=T_id,
    type="FwdFxBuy",
    instrument_identifiers={"Instrument/default/Currency": swap_df['Ccy'].item()},
    transaction_date=str(date(2021,4,1)) ,
    settlement_date=str(date(2021,5,1)),
    units=swap_df['CcyAmount'].item(),
    transaction_price=lm.TransactionPrice(1,'Price'),
    total_consideration=lm.CurrencyAndAmount(swap_df['Ccy2Amount'].item(),swap_df['Ccy2'].item()))
    response= transactions_api.upsert_transactions('FX_FW_Transact',
                                                   Port1_Code,
                                                   transaction_request=[request])
    response= response.to_dict()
    return {'response link':response['links'][2]['href']}

In [16]:
upsert_fx('fx_fwd_sample.csv')

{'response link': 'http://omar.lusid.com/app/insights/logs/0HM7R8JJJG3OE:00000006'}

## 4. Load Fx Forward as Holding

Load FX Forward as holdings- each leg of the forward is loaded in as a tax lot, joined by "Holding/default/ForwardTransactionId" property. This is done using  <a id='https://www.lusid.com/docs/api#operation/AdjustHoldings'>Adjust Holdings</a> you can learn more about the Transaction Portfolios data model <a id='https://www.lusid.com/docs/api#section/Data-Model/Transaction-Portfolios'>here</a>.

In [16]:
class FxForwardHoldingLoader():
    
    def __init__(self, filename,api_factory, scope, portfolio, property_scope):
        self.filename = filename
        self.scope = scope
        self.portfolio = portfolio
        self.property_scope = property_scope
        self.transaction_portfolios_api = api_factory.build(api.TransactionPortfoliosApi)

        fwd_leg_df= pd.read_csv(self.filename)
    
    '''Submit adjust fx forward holdings requests'''
    def adjust_fx_forward_holdings(self, effective_at, adjust_holdings_requests):
        display("-------------")
        display(lusid_response_to_data_frame(adjust_holdings_requests))
        if PUSH_TO_LUSID:
            response = self.transaction_portfolios_api.adjust_holdings(
                scope=self.scope,
                code=self.portfolio,
                effective_at = effective_at,
                adjust_holding_request=adjust_holdings_requests,
                reconciliation_methods=["FxForward"]
            )
            response= response.to_dict()
            display({'response link':response['links'][2]['href']})


    ''' Create fx forward adjust holding requests from a single Tradar entry for a Fwd'''
    def _create_adjust_holdings_requests(self, fx_fwd):
        adjust_holdings_requests = [
            # create dom ccy fwd request
            self._create_adj_holding_request(fx_fwd, True),
            # create dom ccy fgn request
            self._create_adj_holding_request(fx_fwd, False)
        ]
        return adjust_holdings_requests;

    def _create_adj_holding_request(self, fwd_leg_df, is_dom=True):
        ccy = fwd_leg_df["Ccy"] if is_dom else fwd_leg_df["Ccy2"]
        purchase_date = pd.to_datetime(fwd_leg_df['TradeDate'], utc=True, format=DATE_FORMAT).isoformat()
        settlement_date = pd.to_datetime(fwd_leg_df['SettlementDate'], utc=True, format=DATE_FORMAT).isoformat()
        instrument_identifiers = {
            'Instrument/default/Currency': ccy
        }
        properties = {
            # crucial fxforward property to link both legs of fx forward
            "Holding/default/ForwardTransactionId":
                self._create_perp_property("Holding/default/ForwardTransactionId", fwd_leg_df["TradeNumber"])
        }
        tax_lots = [

            # leg
            lm.TargetTaxLotRequest(
                units=(fwd_leg_df["CcyAmount"]*-1) if is_dom else fwd_leg_df["Ccy2Amount"],
                cost=lm.CurrencyAndAmount((fwd_leg_df["CcyAmount"]*-1), fwd_leg_df["Ccy"]) if is_dom else lm.CurrencyAndAmount(
                    fwd_leg_df["Ccy2Amount"], fwd_leg_df["Ccy2"]),
                portfolio_cost=fwd_leg_df["CostFC"],
                price=fwd_leg_df["Price"],
                purchase_date=purchase_date,
                settlement_date=settlement_date,
            )
        ]
        sub_holding_keys=None
        return lm.AdjustHoldingRequest(
            instrument_identifiers,
            sub_holding_keys,
            properties,
            tax_lots
        )

    def _create_perp_property(self, key, value):
        return  lm.PerpetualProperty(key, lm.PropertyValue(label_value=value))

In [17]:
PUSH_TO_LUSID = True
DATE_FORMAT = '%d/%m/%Y'

filename='fx_fwd_sample.csv'
Port_scope='FX_FW_Transact'
Holding_scope='Swap_Holding'

# setup loader
fx_forward_holding_loaders = FxForwardHoldingLoader(filename,
                                                    api_factory, 
                                                    Port_scope, 
                                                    Port1_Code, 
                                                    Holding_scope)
# read in sample fx fwds
fx_fwds = pd.read_csv(filename)

# construct fx fwd requests
effective_at = None
fx_fwd_requests = []
for i, fx_fwd in fx_fwds.iterrows():
    effective_at = str(date(2021,4,5))
    fx_fwd_requests.extend(fx_forward_holding_loaders._create_adjust_holdings_requests(fx_fwd))

# submit fx fwd requests as adjustments (not setting holdings)
fx_forward_holding_loaders.adjust_fx_forward_holdings(effective_at, fx_fwd_requests)

'-------------'

,instrument_identifiers.Instrument/default/Currency,properties.Holding/default/ForwardTransactionId.key,properties.Holding/default/ForwardTransactionId.value.label_value,tax_lots.0.units,tax_lots.0.cost.amount,tax_lots.0.cost.currency,tax_lots.0.portfolio_cost,tax_lots.0.price,tax_lots.0.purchase_date,tax_lots.0.settlement_date
0,GBP,Holding/default/ForwardTransactionId,101,-5000,-5000,GBP,6850,1.37,2021-01-01T00:00:00+00:00,2021-02-01T00:00:00+00:00
1,USD,Holding/default/ForwardTransactionId,101,6850,6850,USD,6850,1.37,2021-01-01T00:00:00+00:00,2021-02-01T00:00:00+00:00


{'response link': 'http://omar.lusid.com/app/insights/logs/0HM7TNLNI8DAJ:00000010'}

## 5 Upsert FX Fwd Qoutes for valuation

Upsert quotes for fwd rates, this will be an input for the valuation

### 5a) Quote FWD Rate

In [32]:
# set up the quotes
effective_date = str(date(2021, 4, 1))
fxfwdSeriesId = lm.QuoteSeriesId(
    provider="Lusid",
    instrument_id="GBP/USD/FxFwdRate/20210501",
    instrument_id_type="LusidInstrumentId",
    quote_type="Price",
    field="mid"
)

FXFWDQuote = lm.QuoteId(
    quote_series_id=fxfwdSeriesId,
    effective_at=effective_date
)

FXFWDQuoteRequest = lm.UpsertQuoteRequest(
    quote_id=FXFWDQuote,
    metric_value=lm.MetricValue(
        value=1.6, unit="USD"),
    lineage="default"
)

response = quotes_api.upsert_quotes(scope="FxData",
                                     request_body={"1": FXFWDQuoteRequest}
                                     )
display(lusid_response_to_data_frame(response))

,response_values
href,None
values.1.quote_id.quote_series_id.provider,Lusid
values.1.quote_id.quote_series_id.price_source,None
values.1.quote_id.quote_series_id.instrument_id,GBP/USD/FxFwdRate/20210501
values.1.quote_id.quote_series_id.instrument_id_type,LusidInstrumentId
values.1.quote_id.quote_series_id.quote_type,Price
values.1.quote_id.quote_series_id.field,mid
values.1.quote_id.effective_at,2021-04-01T00:00:00.0000000+00:00
values.1.metric_value.value,1.6
values.1.metric_value.unit,USD


### 5b) Quote for spot

In [34]:
# set up the quotes
effective_date = str(date(2021, 5, 1))
fxfwdSeriesId = lm.QuoteSeriesId(
    provider="Lusid",
    instrument_id="GBP/USD",
    instrument_id_type="CurrencyPair",
    quote_type="Price",
    field="mid"
)

FXFWDQuote = lm.QuoteId(
    quote_series_id=fxfwdSeriesId,
    effective_at=effective_date
)

FXFWDQuoteRequest = lm.UpsertQuoteRequest(
    quote_id=FXFWDQuote,
    metric_value=lm.MetricValue(
        value=1.41, unit="USD"),
    lineage="default"
)

response = quotes_api.upsert_quotes(scope="FxData",
                                     request_body={"1": FXFWDQuoteRequest}
                                     )
display(lusid_response_to_data_frame(response))

,response_values
href,None
values.1.quote_id.quote_series_id.provider,Lusid
values.1.quote_id.quote_series_id.price_source,None
values.1.quote_id.quote_series_id.instrument_id,GBP/USD
values.1.quote_id.quote_series_id.instrument_id_type,CurrencyPair
values.1.quote_id.quote_series_id.quote_type,Price
values.1.quote_id.quote_series_id.field,mid
values.1.quote_id.effective_at,2021-05-01T00:00:00.0000000+00:00
values.1.metric_value.value,1.41
values.1.metric_value.unit,USD


## 6. Define Recipie for valuation

Define the Recipie that will be used by the Aggregation engine to calculate a valuation for the portfolio.

In [36]:
recipe_scope="Finbourne-Examples"
recipe_code="FxForwards"

config_recipe = lm.ConfigurationRecipe(
        scope=recipe_scope,
        code=recipe_code,
        market=lm.MarketContext(
            market_rules=[
                lm.MarketDataKeyRule(
                    key='Fx.CurrencyPair.*',
                    data_scope='FxData',
                    supplier='Lusid',
                    quote_type='Price',
                    quote_interval='1D.0D',
                    field='close'),
                lm.MarketDataKeyRule(
                    key='Fx.*.*',
                    data_scope='FxData',
                    supplier='Lusid',
                    quote_type='Price',
                    quote_interval='1D.0D',
                    field='mid'),
                  lm.MarketDataKeyRule(
                    key='Equity.*.*',
                    data_scope='FxData',
                    supplier='Lusid',
                    quote_type='Price',
                    quote_interval='1D.0D',
                    field='mid'), 
            ],
            suppliers=lm.MarketContextSuppliers(
                commodity='Lusid',
                credit='Lusid',
                equity='Lusid',
                fx='Lusid',
                rates='Lusid'),
            options=lm.MarketOptions(
                default_supplier='Lusid',
                default_instrument_code_type='Figi',
                default_scope='Lusid',
                attempt_to_infer_missing_fx=True
            ),
        ),
        pricing=lm.PricingContext(
            options=lm.PricingOptions(
                model_selection=lm.ModelSelection(
                    library="Lusid",
                    model="ForwardSpecifiedRateUndiscounted"
                ),
                use_instrument_type_to_determine_pricer=False,
                allow_any_instruments_with_sec_uid_to_price_off_lookup=False,
                produce_separate_result_for_linear_otc_legs=True,
                allow_partially_successful_evaluation=False,
                enable_use_of_cached_unit_results=False,
                window_valuation_on_instrument_start_end=False,
                remove_contingent_cashflows_in_payment_diary=False,
                use_child_sub_holding_keys_for_portfolio_expansion=False
            )),
        description="ifrs test"
    )

response = configuration_recipe_api.upsert_configuration_recipe(
    lm.UpsertRecipeRequest(configuration_recipe=config_recipe)
)
display(lusid_response_to_data_frame(response))

,response_values
href,None
value,2021-04-09 13:29:17.265485+00:00
links.0.relation,RequestLogs
links.0.href,http://omar.lusid.com/app/insights/logs/0HM7R8...
links.0.description,A link to the LUSID Insights website showing a...
links.0.method,GET


## 7. Run Valuation

Construct a valuation request specifying which metrics we would like included. This request is then passed to the Lusid aggregation API.

In [39]:
recipe_scope="Finbourne-Examples"
recipe_code="FxForwards"

valuation_date = str(date(2021, 4, 1))

# Create the valuation request
valuation_request = lm.ValuationRequest(
    recipe_id= lm.ResourceId(scope= recipe_scope, code=recipe_code),
    metrics=[
        lm.AggregateSpec("Instrument/default/Name", "Sum"),
        lm.AggregateSpec("Holding/default/Type", "Value"),
        lm.AggregateSpec("Holding/default/PV", "Sum"),
    ],
    group_by=["Instrument/default/Name","Holding/default/Type"],
    portfolio_entity_ids=[lm.PortfolioEntityId(
        scope='FX_FW_Transact', 
        code=Port1_Code)
                         ],
    valuation_schedule=lm.ValuationSchedule(effective_at=valuation_date)
)

# Perform a valuation
valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request)

#print(valuation)
Valuations = pd.DataFrame(valuation.data)

## Output

Returns the valuation metrics as a dataframe. PV of the forward position is shown in forward space (valuation is converted back to GBP using the fwd rate).

In [40]:
Valuations

,Sum(Instrument/default/Name),Holding/default/Type,Sum(Holding/default/PV)
0,GBP,Balance,10000.00
1,GBP,ForwardFx,718.75
